In [1]:
try:
    from medcat.cat import CAT
except:
    print("WARNING: Runtime will restart automatically and please run other cells thereafter.")
    exit()

/home/vmadmin/medcat_venv/lib/python3.8/site-packages/medcat/cat.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import pandas as pd
import numpy as np
import json
import os
import spacy
import logging

from medcat.cat import CAT
from medcat.cdb import CDB
from medcat.config import Config
from medcat.vocab import Vocab
from medcat.meta_cat import MetaCAT
from medcat.preprocessing.tokenizers import TokenizerWrapperBPE
from tokenizers import ByteLevelBPETokenizer
import dask.dataframe as dd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

In [3]:
#'increase size of notebook'
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
# os.chdir('/content/drive/My Drive/Colab Notebooks/mimic-iii-clinical-database-1.4/src')
path = "/home/vmadmin/Anchor_and_Span/src/"
if os.getcwd() != path:
    os.chdir(path)

In [5]:
!ls .

__pycache__	explore.ipynb	  mydask.png	   test_linking.py
anchor_span.py	medcat_ann.ipynb  scisp_ann.ipynb  utils.py
annotate.py	mstext_ann.ipynb  scisp_ann.py	   visualize_annotations.ipynb
creds		mstext_ann.py	  subscription


In [6]:
data_read_dask = dd.read_csv('../../mimic-iii/cleaned/notes/NOTESEVENTS_0.csv', 
                        encoding="utf-8", engine="python", dtype={"CLEANED_TEXT": str})
data_read_dask.head(n=2, compute=True)

index  \
0      0   
1      1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [7]:
nlp = spacy.load('en_core_web_sm')

In [8]:
#using spacy multiprocess pip function to multiprocess data
def spacyProcess(data, dask=False):
    if dask:
        data.loc[:,"PROCESSED_TEXT"] = data['CLEANED_TEXT'].apply(lambda x:nlp(x))
        return data
    documents = nlp.pipe(data['CLEANED_TEXT'].tolist())
    return documents

In [9]:
start, end = 0, 10

In [10]:
data_read_ = data_read_dask.compute()

In [11]:
"data_read_das is of type {} and data_read is of type {}".format(type(data_read_dask), type(data_read_))

"data_read_das is of type <class 'dask.dataframe.core.DataFrame'> and data_read is of type <class 'pandas.core.frame.DataFrame'>"

In [12]:
if len(data_read_) < end:
    end = len(data_read_)
data_read = data_read_[start:end]

In [13]:
data_read.shape

(10, 2)

In [14]:
spacy_processed_data = spacyProcess(data=data_read)

In [15]:
document_sent_map = {}
sentences = []
sent_id = 0
for doc_id,doc in enumerate(spacy_processed_data):
    doc_sents = doc.sents
    doc_sents_ids = []
    for sent in doc_sents:
        sentences.append(" ".join([m.text for m in sent]))
        doc_sents_ids.append(sent_id)
        sent_id += 1
    document_sent_map[doc_id] = (doc_sents_ids[0], doc_sents_ids[-1]+1)
data_read_sp = pd.DataFrame({'CLEANED_TEXT':sentences})

In [16]:
dict([[k,v] for k,v in document_sent_map.items()][:5])

{0: (0, 11), 1: (11, 184), 2: (184, 304), 3: (304, 510), 4: (510, 706)}

In [17]:
data_read_sp.head()

,CLEANED_TEXT
0,"Admission Date : 2151 - 7 - 16 Discharge Date : 2151 - 8 - 4 Service : ADDENDUM : RADIOLOGIC STUDIES : Radiologic studies also included a chest CT , which confirmed cavitary lesions in the left lung apex consistent with infectious process / tuberculosis ."
1,This also moderate - sized left pleural effusion .
2,HEAD CT :
3,"Head CT showed no intracranial hemorrhage or mass effect , but old infarction consistent with past medical history ."
4,ABDOMINAL CT :


In [18]:
# format the df to match: required input data for multiprocessing = [(doc_id, doc_text), (doc_id, doc_text), ...]
def data_iterator_df(data):
    for id, row in data[['CLEANED_TEXT']].iterrows():
        yield (id, str(row['CLEANED_TEXT']))

In [19]:
cat = CAT.load_model_pack("../models/umls_sm_pt2ch_533bab5115c6c2d6.zip")

2023-07-18 19:10:16,355 - Found an existing unziped model pack at: ../models/umls_sm_pt2ch_533bab5115c6c2d6, the provided zip will not be touched.
2023-07-18 19:10:16,362 - Loading model pack with dill format
2023-07-18 19:10:16,363 - Reading CDB data from ../models/umls_sm_pt2ch_533bab5115c6c2d6/cdb.dat
2023-07-18 19:10:37,443 - You have MedCAT version '1.8.0' installed while the CDB was exported by MedCAT version '1.3.1.dev315',
which may or may not work. If you experience any compatibility issues, please reinstall MedCAT
or download the compatible model.
/home/vmadmin/medcat_venv/lib/python3.8/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.6.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.

In [20]:
# Set a batch size to control for the variablity between document sizes
batch_size_chars = 500000 # Batch size (BS) in number of characters

# Run model
results = cat.multiprocessing(data_iterator_df(data_read_sp),  # Formatted data
                              # batch_size_chars = batch_size_chars,
                              nproc=8) # Number of processors

2023-07-18 19:11:00,307 - Annotated until now: 0 docs; Current BS: 1172 docs; Elapsed time: 0.00 minutes


In [21]:
len(results)

1172

In [22]:
#find the token id/position of an entity given its char offset from start of a sentence and the sentence
def fetch_entitis_span_pos(entity, sentence, extractor):
    sentence_tokenized = dict([(i, j) for i, j in enumerate(sentence.split())])
    token_ids_cumulative_length = {}
    if extractor == 'MSTA4H':
        char_offset, entity_len, entity_text = entity.offset, entity.length, entity.text
    if extractor == 'MEDCAT':
        char_offset, entity_len, entity_text = entity[0], entity[1]-entity[0], entity[2]
    curr_offset = 0
    span, end_span_found = [], False
    if char_offset == 0:
        span.append(0)

    for i, j in sentence_tokenized.items():
        if curr_offset >= char_offset + entity_len:
            span.append(i)
            end_span_found = True
        else:
            curr_offset = curr_offset + len(j) + 1
            if curr_offset == char_offset:
                span.append(i + 1)
        if end_span_found == True:
            if len(span) != 2:
                print("Failed to detect span position {} {}".format(entity_text, span))
            try:
                assert len(span) == 2
                break
            except Exception as e:
                return entity_text, span
    return entity_text, span

In [23]:
results_copy = dict(sorted(results.items(), key=lambda x:x[0]))

In [27]:
dataset_ann = []
for doc_id, document_sent_range in document_sent_map.items():
    doc_ann = {}
    doc_entities = []
    sents = []
    for sent_id, doc in enumerate(results_copy):
        if sent_id in range(document_sent_range[0], document_sent_range[-1]):
            sents.append(sentences[sent_id].split())
            for k,v in results_copy[doc]['entities'].items():
                entity = {}
                _entity_ = (v['start'], v['end'], v['source_value'])
                entity_text, entity_span_pos = fetch_entitis_span_pos(_entity_, sentences[sent_id], "MEDCAT")
                try:
                    assert len(entity_span_pos) == 2
                except AssertionError:
                    pass
                entity['name'] = entity_text
                entity['sent_id'] = sent_id
                entity['char_position'] = [v['start'], v['end']]
                entity['token_pos'] = entity_span_pos
                entity['score'] = v['acc']
                entity['linked_entities'] = []
                linked_entity = {}
                for m,n in zip(v['types'], v['type_ids']):
                    linked_entity['cui'] = v['cui']
                    linked_entity['type'] = m
                    linked_entity['type_id'] = n
                    entity['linked_entities'].append(linked_entity)
                if entity:
                    doc_entities.append(entity)
        if sent_id == (document_sent_range[-1]-1):
            break
    doc_ann['Entities'] = doc_entities
    doc_ann['Sents'] = sents
    dataset_ann.append(doc_ann)

Failed to detect span position 25PM [22]
Failed to detect span position 30AM [20]
Failed to detect span position 26PM [9]
Failed to detect span position 17AM [9]
Failed to detect span position 26PM [18]
Failed to detect span position 26PM [9]
Failed to detect span position 26PM [18]
Failed to detect span position 11PM [12]
Failed to detect span position 26PM [31]
Failed to detect span position 26PM [39]
Failed to detect span position 26PM [52]
Failed to detect span position 11PM [7]
Failed to detect span position 32PM [8]
Failed to detect span position 5 - mm [24]
Failed to detect span position 13PM [25]
Failed to detect span position 13PM [9]
Failed to detect span position 13PM [18]
Failed to detect span position 13PM [27]
Failed to detect span position 13PM [37]


In [28]:
dataset_ann

[{'Entities': [{'name': 'chest',
    'sent_id': 0,
    'char_position': [138, 143],
    'token_pos': [28, 29],
    'score': 0.8211847196206128,
    'linked_entities': [{'cui': 'C0817096',
      'type': 'Body Location or Region',
      'type_id': 'T029'}]},
   {'name': 'lesions',
    'sent_id': 0,
    'char_position': [174, 181],
    'token_pos': [34, 35],
    'score': 0.8631982261939173,
    'linked_entities': [{'cui': 'C0221198',
      'type': 'Finding',
      'type_id': 'T033'}]},
   {'name': 'lung',
    'sent_id': 0,
    'char_position': [194, 198],
    'token_pos': [38, 39],
    'score': 0.4046709274009144,
    'linked_entities': [{'cui': 'C0024109',
      'type': 'Body Part, Organ, or Organ Component',
      'type_id': 'T023'}]},
   {'name': 'infectious process',
    'sent_id': 0,
    'char_position': [220, 238],
    'token_pos': [42, 44],
    'score': 1.0,
    'linked_entities': [{'cui': 'C0745283',
      'type': 'Pathologic Function',
      'type_id': 'T046'}]},
   {'name': 'tub

In [24]:
def createDir(path):
    dest = path
    if not os.path.exists(path):
        dest = os.makedirs(path)
    return dest

In [25]:
dest_dir = '../anns/medcat/'
createDir(dest_dir)

'../anns/medcat/'

In [26]:
import pickle

annotation_file = 'anns_mult_{}_{}.pkl'.format(start, end)
with open(os.path.join(dest_dir, annotation_file), 'wb') as a:
    pickle.dump(dataset_ann, a, protocol=pickle.HIGHEST_PROTOCOL)
    a.close()